In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import cv2
import sys

In [2]:
## Annotation set
anno_set = 'Set 1/'

## Names of files where mapping is being performed
file1 = '20230313_SE_Lek1_P2D3_DJI_0312'
file2 = '20230313_SE_Lek1_P3D5_DJI_0575'

## Names of files compressed
f1 = 'P2D3_DJI_0312'
f2 = 'P3D5_DJI_0575'

## Column names used often
start_frame1 = 'start_frame_' + f1
start_frame2 = 'start_frame_' + f2
end_frame1 = 'end_frame_' + f1
end_frame2 = 'end_frame_' + f2
Frame1 = 'Frame_' + f1
Frame2 = 'Frame_' + f2
ID1 = 'ID_' + f1
ID2 = 'ID_' + f2

In [3]:
## Read dataframes for both files
df1 = pd.read_csv('/Users/vivekhsridhar/Library/Mobile Documents/com~apple~CloudDocs/Documents/Code/Python/OpenCV/plot_tracks/data/' + file1 + '_tracks.csv')
df1 = df1.loc[:,['frame', 'id', 'bb_left', 'bb_top', 'bb_width', 'bb_height']]
df2 = pd.read_csv('/Users/vivekhsridhar/Library/Mobile Documents/com~apple~CloudDocs/Documents/Code/Python/OpenCV/plot_tracks/data/' + file2 + '_tracks.csv')
df2 = df2.loc[:,['frame', 'id', 'bb_left', 'bb_top', 'bb_width', 'bb_height']]

## Read the manually created mapping file
map_df = pd.read_csv('/Users/vivekhsridhar/Library/Mobile Documents/com~apple~CloudDocs/Documents/Manuscripts/10_dataset/Data/' + anno_set + 'Mapping_' + file1 + '_' + file2 + '.csv', sep=';')
map_df.columns = [Frame1, ID1, Frame2, ID2]

## Frame mapping for overlap (vid2 - vid1) e.g. vid2(399) == vid1(396)
lag = 3

In [4]:
## Incorporate lag in the tracking data
if lag > 0:
    df1['lagged_frame'] = df1['frame']
    df2['lagged_frame'] = df2['frame'] + lag
elif lag < 0:
    df1['lagged_frame'] = df1['frame'] + lag
    df2['lagged_frame'] = df2['frame']

In [5]:
## Function to find min and max in an array within the 'current chain'
def find_chain_min_max(array, element):
    ## Find the index of the given element in the array
    index = np.where(array == element)[0][0]
    
    ## Find the starting index of the consecutive chain
    start_index = index
    while start_index > 0 and array[start_index] - array[start_index - 1] == 1:
        start_index -= 1
    
    ## Find the ending index of the consecutive chain
    end_index = index
    while end_index < len(array) - 1 and array[end_index + 1] - array[end_index] == 1:
        end_index += 1
    
    ## Return the minimum and maximum within the consecutive chain
    return array[start_index], array[end_index]

## Function to calculate overlap frames
def calculate_overlap(row):
    start_overlap = max(row[start_frame1], row[start_frame2])
    end_overlap = min(row[end_frame1], row[end_frame2])
    if start_overlap <= end_overlap:
        return pd.Series([start_overlap, end_overlap])
    else:
        return pd.Series([None, None])

### Make a dataframe that has the same rows as  'map_df' but includes start and end frames of each ID in that column

In [6]:
## Ready new mapping dataframe with start and end frames for each ID
for index, row in map_df.iterrows():
    ## Get tracklets from current row
    tracklet1 = df1[df1['id'] == row[ID1]]
    tracklet2 = df2[df2['id'] == row[ID2]]
    
    ## Get start and end frame of tracklets based on annotated frame
    anno_fr1 = row[Frame1]
    anno_fr2 = row[Frame2]
    
    start_fr1, end_fr1 = find_chain_min_max(np.array(tracklet1['frame']), anno_fr1)
    start_fr2, end_fr2 = find_chain_min_max(np.array(tracklet2['frame']), anno_fr2)
    
    ## Compile modified dataframe
    tmp = pd.DataFrame([start_fr1, end_fr1, row[ID1], start_fr2, end_fr2, row[ID2]]).T
    tmp.columns = [start_frame1, end_frame1, ID1, start_frame2, end_frame2, ID2]
    if index != 0:
        df = pd.concat((df, tmp), axis=0)
    else:
        df = tmp
        
df = df.reset_index(drop = True)
df = df.drop_duplicates()

Errors detected were manually deleted in the mapping file. Below, I list the errors I found during my process of checking mapping quality.
1. ID 77 in P2D3_0312 mapped to ID 44 in P3D5_0575 
2. ID 161 in P1D1_0294 mapped to ID 65 in P2D3_0310
3. ID 130 in P2D4_0940 mapped to ID 85 in P3D6_0923
4. ID 137 in P2D4_0940 mapped to ID 86 in P3D6_0923
5. ID 11 in P2D4_0940 mapped to ID 87 in P3D6_0923

These rows were deleted because the frame indicated in the map_df dataframe was outside of the life of the corresponding ID.
I also manually added lines at the end of the mapping file.

In [7]:
## Calculate overlap between the two videos
overlap_frames = df.apply(calculate_overlap, axis=1)
overlap_frames.columns = ['start_frame_overlap', 'end_frame_overlap']

## Add the calculated overlap frames to the original DataFrame
df = pd.concat([df, overlap_frames], axis=1)

## Apply lag to the overlap window as appropriate
## Here, we assume that vid1 needs addition of 'lag' frames to match frame numbers with vid2
## If start_frame_vid1 >= start_frame_vid2: start_frame_lag = start_frame_overlap + lag
df['start_frame_lag'] = df['start_frame_overlap']
df.loc[df[start_frame1] >= df[start_frame2], 'start_frame_lag'] += 3
df['end_frame_lag'] = df['end_frame_overlap']
df.loc[df[end_frame1] >= df[end_frame2], 'end_frame_lag'] -= 3
df['lag_overlap_duration'] = df['end_frame_lag'] - df['start_frame_lag'] + 1
df.head()

,start_frame_P2D3_DJI_0312,end_frame_P2D3_DJI_0312,ID_P2D3_DJI_0312,start_frame_P3D5_DJI_0575,end_frame_P3D5_DJI_0575,ID_P3D5_DJI_0575,start_frame_overlap,end_frame_overlap,start_frame_lag,end_frame_lag,lag_overlap_duration
0,0.0,7.0,61.0,0.0,840.0,21.0,0.0,7.0,3.0,7.0,5.0
1,0.0,32.0,55.0,0.0,5813.0,18.0,0.0,32.0,3.0,32.0,30.0
2,0.0,72.0,8.0,0.0,3238.0,11.0,0.0,72.0,3.0,72.0,70.0
3,0.0,1503.0,7.0,0.0,5813.0,10.0,0.0,1503.0,3.0,1503.0,1501.0
4,1509.0,5814.0,136.0,0.0,5813.0,10.0,1509.0,5813.0,1512.0,5810.0,4299.0


### Plot videos of the two videos with colours to indicate corresponding tracklets in the two videos

In [8]:
## Get paths and parameters for creating the output videos
input_vidpath1 = '/Users/vivekhsridhar/Library/Mobile Documents/com~apple~CloudDocs/Documents/Code/Python/OpenCV/plot_tracks/data/' + file1 + '.MP4'
input_vidpath2 = '/Users/vivekhsridhar/Library/Mobile Documents/com~apple~CloudDocs/Documents/Code/Python/OpenCV/plot_tracks/data/' + file2 + '.MP4'
output_vidpath1 = '/Users/vivekhsridhar/Library/Mobile Documents/com~apple~CloudDocs/Documents/Code/Python/OpenCV/plot_tracks/output/tracks/' + anno_set + file1 + '_frameno.mp4'
output_vidpath2 = '/Users/vivekhsridhar/Library/Mobile Documents/com~apple~CloudDocs/Documents/Code/Python/OpenCV/plot_tracks/output/tracks/' + anno_set + file2 + '_frameno.mp4'
output_vidpath_merged = '/Users/vivekhsridhar/Library/Mobile Documents/com~apple~CloudDocs/Documents/Code/Python/OpenCV/plot_tracks/output/tracks/' anno_set + file1 + '_' + file2 + '_merged.mp4'
codec = 'DIVX'
scaling = 0.5

## Plotting parameters
max_radius = 5
frame_range = 100
col_mult = int(255/len(df))
colours = [(i*col_mult,255,255) for i in range(len(df))]

## Include the colour column to df
df['colour'] = colours

In [9]:
## Calculate positions of all individuals in both videos
df1['pos_x'] = (df1['bb_left'] + df1['bb_width']/2)*scaling
df1['pos_y'] = (df1['bb_top'] + df1['bb_height']/2)*scaling
df2['pos_x'] = (df2['bb_left'] + df2['bb_width']/2)*scaling
df2['pos_y'] = (df2['bb_top'] + df2['bb_height']/2)*scaling

In [10]:
## Write output video 1
## Open video
cap = cv2.VideoCapture(input_vidpath1)
if cap.isOpened() == False:
    sys.exit('Video file cannot be read! Please check input_vidpath to ensure it is correctly pointing to the video file')

## Video writer class to output video with contour and centroid of tracked object(s)
# make sure the frame size matches size of array 'final'
fourcc = cv2.VideoWriter_fourcc(*codec)
output_framesize = (int(cap.read()[1].shape[1]*scaling),int(cap.read()[1].shape[0]*scaling))
out = cv2.VideoWriter(filename = output_vidpath1, fourcc = fourcc, fps = 30.0, frameSize = output_framesize, isColor = True)

last = 0

while(True):
    ## Capture frame-by-frame
    ret, frame = cap.read()

    this = cap.get(1)
    
    if ret == True:
        ## Preprocess the image for background subtraction
        frame = cv2.resize(frame, None, fx=scaling, fy=scaling, interpolation=cv2.INTER_LINEAR)
        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
        
        ## Filter objects of interest based on the current frame
#         if lag > 0:
#             filtered_df = df[(df['start_frame_lag'] <= this) & (df['end_frame_overlap'] >= this)]
#         else:
#             filtered_df = df[(df['start_frame_overlap'] <= this) & (df['end_frame_lag'] >= this)]
#         if filtered_df.empty == False:
#             indices, interested_objects = zip(*[(idx, row[ID1]) for idx, row in filtered_df.iterrows()])

#             ## Filter MOT dataframe based on the current frame and interested objects
#             tmp = df1[(df1['frame'] == this) & (df1['id'].isin(interested_objects))]
#             tmp = tmp.reset_index()
            
#             if len(filtered_df) != len(tmp):
#                 print('no')
#                 break
            
#             for i in range(0,len(tmp)):
#                 x = int(tmp.loc[i,'pos_x'])
#                 y = int(tmp.loc[i,'pos_y'])
#                 r = int(max_radius - (this - tmp.loc[i,'frame'])//(frame_range/max_radius))
#                 c = df.loc[indices[np.where(interested_objects == tmp.loc[i,'id'])[0][0]], 'colour']
#                 cv2.circle(frame, (x,y), r, c, -1, cv2.LINE_AA)
#                 cv2.putText(frame, str(int(tmp.loc[i,'id'])), (x+20,y-20), cv2.FONT_HERSHEY_PLAIN, 2, (0, 0, 255), 2)

        cv2.putText(frame, str(this), (20,50), cv2.FONT_HERSHEY_PLAIN, 2, (0, 0, 255), 2)
        frame = cv2.cvtColor(frame, cv2.COLOR_HSV2BGR)

        ## Display the resulting frame
        out.write(frame)
        cv2.imshow('frame', frame)
        if cv2.waitKey(1) == 27:
            break
            
    if last >= this:
        break
    
    last = this

## When everything done, release the capture
cap.release()
out.release()
cv2.destroyAllWindows()
cv2.waitKey(1)

OpenCV: FFMPEG: tag 0x58564944/'DIVX' is not supported with codec id 12 and format 'mp4 / MP4 (MPEG-4 Part 14)'
OpenCV: FFMPEG: fallback to use tag 0x7634706d/'mp4v'


-1

In [16]:
## Write output video 2
## Open video
cap = cv2.VideoCapture(input_vidpath2)
if cap.isOpened() == False:
    sys.exit('Video file cannot be read! Please check input_vidpath to ensure it is correctly pointing to the video file')

## Video writer class to output video with contour and centroid of tracked object(s)
# make sure the frame size matches size of array 'final'
fourcc = cv2.VideoWriter_fourcc(*codec)
output_framesize = (int(cap.read()[1].shape[1]*scaling),int(cap.read()[1].shape[0]*scaling))
out = cv2.VideoWriter(filename = output_vidpath2, fourcc = fourcc, fps = 30.0, frameSize = output_framesize, isColor = True)

last = 0

while(True):
    ## Capture frame-by-frame
    ret, frame = cap.read()

    this = cap.get(1)
    
    if ret == True:
        ## Preprocess the image for background subtraction
        frame = cv2.resize(frame, None, fx=scaling, fy=scaling, interpolation=cv2.INTER_LINEAR)
        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
        
        ## Filter objects of interest based on the current frame
#         if lag > 0:
#             filtered_df = df[(df['start_frame_overlap'] <= this) & (df['end_frame_lag'] >= this)]
#         else:
#             filtered_df = df[(df['start_frame_lag'] <= this) & (df['end_frame_overlap'] >= this)]
#         if filtered_df.empty == False:
#             indices, interested_objects = zip(*[(idx, row[ID2]) for idx, row in filtered_df.iterrows()])

#             ## Filter MOT dataframe based on the current frame and interested objects
#             tmp = df2[(df2['frame'] == this) & (df2['id'].isin(interested_objects))]
#             tmp = tmp.reset_index()

#             for i in range(0,len(tmp)):
#                 x = int(tmp.loc[i,'pos_x'])
#                 y = int(tmp.loc[i,'pos_y'])
#                 r = int(max_radius - (this - tmp.loc[i,'frame'])//(frame_range/max_radius))
#                 c = df.loc[indices[np.where(interested_objects == tmp.loc[i,'id'])[0][0]], 'colour']
#                 cv2.circle(frame, (x,y), r, c, -1, cv2.LINE_AA)
#                 cv2.putText(frame, str(int(tmp.loc[i,'id'])), (x+20,y-20), cv2.FONT_HERSHEY_PLAIN, 2, (0, 0, 255), 2)

        cv2.putText(frame, str(this), (20,50), cv2.FONT_HERSHEY_PLAIN, 2, (0, 0, 255), 2)
        frame = cv2.cvtColor(frame, cv2.COLOR_HSV2BGR)

        ## Display the resulting frame
        out.write(frame)
        cv2.imshow('frame', frame)
        if cv2.waitKey(1) == 27:
            break
            
    if last >= this:
        break
    
    last = this

## When everything done, release the capture
cap.release()
out.release()
cv2.destroyAllWindows()
cv2.waitKey(1)

OpenCV: FFMPEG: tag 0x58564944/'DIVX' is not supported with codec id 12 and format 'mp4 / MP4 (MPEG-4 Part 14)'
OpenCV: FFMPEG: fallback to use tag 0x7634706d/'mp4v'


-1

In [11]:
## Write the concatenated video
## Open the two videos
cap1 = cv2.VideoCapture(output_vidpath1)
if not cap1.isOpened():
    sys.exit('Video file 1 cannot be read! Please check input_vidpath1 to ensure it is correctly pointing to the video file')

cap2 = cv2.VideoCapture(output_vidpath2)
if not cap2.isOpened():
    sys.exit('Video file 2 cannot be read! Please check input_vidpath2 to ensure it is correctly pointing to the video file')

## Video writer class to output concatenated video
fourcc = cv2.VideoWriter_fourcc(*codec)
frame_width = int(cap1.get(cv2.CAP_PROP_FRAME_WIDTH) * scaling)
frame_height = int(cap1.get(cv2.CAP_PROP_FRAME_HEIGHT) * scaling)
output_framesize = (frame_width, frame_height * 2)
out = cv2.VideoWriter(filename=output_vidpath_merged, fourcc=fourcc, fps=30.0, frameSize=output_framesize, isColor=True)

## Create a black frame for the delayed video
black_frame = np.zeros((frame_height, frame_width, 3), dtype=np.uint8)

## Buffer to store initial frames of the delayed video
buffer = []
buffer_size = abs(lag)  # Adjust buffer size based on the absolute value of lag

## Pre-fill buffer with initial frames of Video 2 if lag is positive, else pre-fill buffer with Video 1 frames
if lag < 0:
    for _ in range(buffer_size):
        ret2, frame2 = cap2.read()
        if not ret2:
            break
        frame2 = cv2.resize(frame2, (frame_width, frame_height), interpolation=cv2.INTER_LINEAR)
        buffer.append(frame2)
elif lag > 0:
    for _ in range(buffer_size):
        ret1, frame1 = cap1.read()
        if not ret1:
            break
        frame1 = cv2.resize(frame1, (frame_width, frame_height), interpolation=cv2.INTER_LINEAR)
        buffer.append(frame1)

## Loop through frames of both videos simultaneously and concatenate vertically
frame_counter = 0
while True:
    if lag < 0:
        ret1, frame1 = cap1.read()
        if not ret1:
            frame1 = black_frame
        else:
            frame1 = cv2.resize(frame1, (frame_width, frame_height), interpolation=cv2.INTER_LINEAR)
            if frame_counter < buffer_size:
                frame2 = black_frame
            else:
                frame2 = buffer.pop(0)
                ret2, new_frame2 = cap2.read()
                if ret2:
                    new_frame2 = cv2.resize(new_frame2, (frame_width, frame_height), interpolation=cv2.INTER_LINEAR)
                    buffer.append(new_frame2)
                else:
                    frame2 = black_frame
    elif lag > 0:
        ret2, frame2 = cap2.read()
        if not ret2:
            frame2 = black_frame
        else:
            frame2 = cv2.resize(frame2, (frame_width, frame_height), interpolation=cv2.INTER_LINEAR)
            if frame_counter < buffer_size:
                frame1 = black_frame
            else:
                frame1 = buffer.pop(0)
                ret1, new_frame1 = cap1.read()
                if ret1:
                    new_frame1 = cv2.resize(new_frame1, (frame_width, frame_height), interpolation=cv2.INTER_LINEAR)
                    buffer.append(new_frame1)
                else:
                    frame1 = black_frame

    concatenated_frame = cv2.vconcat([frame1, frame2])

    out.write(concatenated_frame)
    cv2.imshow('Concatenated Frame', concatenated_frame)
    if cv2.waitKey(1) == 27:
        break

    frame_counter += 1

## When everything done, release the capture
cap1.release()
cap2.release()
out.release()
cv2.destroyAllWindows()
cv2.waitKey(1)

OpenCV: FFMPEG: tag 0x58564944/'DIVX' is not supported with codec id 12 and format 'mp4 / MP4 (MPEG-4 Part 14)'
OpenCV: FFMPEG: fallback to use tag 0x7634706d/'mp4v'


-1